In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
from torch.utils.data import DataLoader
from typing import Tuple, Dict, Any
import time

# Verifica disponibilità GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizzato: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Dispositivo utilizzato: cpu


In [2]:
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader, Subset
import torch
from typing import Tuple
import numpy as np

class DataManager:
    """Classe per gestire il caricamento e preprocessing dei dati MNIST"""
    
    def __init__(self, batch_size: int = 128, test_batch_size: int = 64, subset_ratio: float = 0.2):
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.subset_ratio = subset_ratio
        
        # Trasformazioni per compatibilità con DenseNet (richiede 3 canali)
        # Utilizziamo 112x112 per un buon bilanciamento tra qualità e prestazioni
        self.transform_train = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((112, 112)),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet stats
        ])
        
        self.transform_test = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((112, 112)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        self._load_data()
    
    def _load_data(self):
        """Carica i dataset MNIST"""
        self.trainset = torchvision.datasets.MNIST(
            root='./data', train=True, download=True, transform=self.transform_train
        )
        self.testset = torchvision.datasets.MNIST(
            root='./data', train=False, download=True, transform=self.transform_test
        )
        
        # Utilizza un subset per accelerare il prototipaggio
        if self.subset_ratio < 1.0:
            # Subset training bilanciato
            train_size = int(len(self.trainset) * self.subset_ratio)
            train_indices = np.random.choice(len(self.trainset), train_size, replace=False)
            self.trainset = Subset(self.trainset, train_indices)
            
            # Subset test proporzionale
            test_size = int(len(self.testset) * (self.subset_ratio * 2))  # Più campioni per test affidabile
            test_indices = np.random.choice(len(self.testset), test_size, replace=False)
            self.testset = Subset(self.testset, test_indices)
        
        # DataLoader
        self.trainloader = DataLoader(
            self.trainset, 
            batch_size=self.batch_size, 
            shuffle=True, 
            num_workers=0,
            pin_memory=torch.cuda.is_available()
        )
        
        self.testloader = DataLoader(
            self.testset, 
            batch_size=self.test_batch_size, 
            shuffle=False, 
            num_workers=0,
            pin_memory=torch.cuda.is_available()
        )
        
        print(f"Dataset caricato: {len(self.trainset):,} train, {len(self.testset):,} test")
    
    def get_sample_batch(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """Restituisce un batch di esempio per testing"""
        return next(iter(self.testloader))

# Inizializza il data manager
data_manager = DataManager()
trainloader = data_manager.trainloader
testloader = data_manager.testloader

Dataset caricato: 60000 train, 10000 test


In [ ]:
class MNISTClassifier:
    """Classe per il classificatore MNIST basato su DenseNet121 pre-addestrato"""
    
    def __init__(self, num_classes: int = 10):
        self.num_classes = num_classes
        self.device = device
        self._build_model()
        
    def _build_model(self):
        """Costruisce il modello con DenseNet121 pre-addestrato"""
        self.model = models.densenet121(pretrained=True)
        
        # Congela tutti i layer tranne l'ultimo
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Sostituisce il classificatore per MNIST (10 classi)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(in_features, self.num_classes)
        
        # Solo l'ultimo layer è trainable
        for param in self.model.classifier.parameters():
            param.requires_grad = True
            
        self.model = self.model.to(self.device)
        print(f"Modello creato con {in_features} -> {self.num_classes} neuroni nel classificatore")
        
    def fine_tune(self, trainloader: DataLoader, epochs: int = 2, lr: float = 0.001):
        """Fine-tuning dell'ultimo layer"""
        print(f"Inizio fine-tuning per {epochs} epoche...")
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.classifier.parameters(), lr=lr)
        
        # Abilita mixed precision se GPU disponibile
        use_amp = torch.cuda.is_available()
        if use_amp:
            scaler = torch.cuda.amp.GradScaler()
        
        self.model.train()
        
        for epoch in range(epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            start_time = time.time()
            
            for i, (inputs, labels) in enumerate(trainloader):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                
                # Forward pass con mixed precision se disponibile
                if use_amp:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(inputs)
                        loss = criterion(outputs, labels)
                    
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                if i % 100 == 99:  # Progress ogni 100 batch
                    print(f'[Epoca {epoch+1}, Batch {i+1}] Loss: {running_loss/100:.3f}, '
                          f'Accuracy: {100*correct/total:.2f}%')
                    running_loss = 0.0
            
            epoch_time = time.time() - start_time
            epoch_acc = 100 * correct / total
            print(f'Epoca {epoch+1} completata in {epoch_time:.1f}s - Accuracy: {epoch_acc:.2f}%')
        
        print("Fine-tuning completato!")
        
    def evaluate(self, testloader: DataLoader) -> float:
        """Valuta l'accuracy del modello sul test set"""
        self.model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                # Mixed precision anche per inference se disponibile
                if torch.cuda.is_available():
                    with torch.cuda.amp.autocast():
                        outputs = self.model(inputs)
                else:
                    outputs = self.model(inputs)
                    
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Accuracy sul test set: {accuracy:.2f}% ({correct}/{total})')
        return accuracy
    
    def get_model(self):
        """Restituisce il modello PyTorch"""
        return self.model

# Crea e addestra il classificatore
classifier = MNISTClassifier()

# Prima verifichiamo l'accuracy senza fine-tuning
print("=== ACCURACY PRIMA DEL FINE-TUNING ===")
accuracy_before = classifier.evaluate(testloader)

# Esegui fine-tuning
print("\n=== FINE-TUNING ===")
classifier.fine_tune(trainloader, epochs=2)

# Verifica accuracy dopo fine-tuning
print("\n=== ACCURACY DOPO IL FINE-TUNING ===")
accuracy_after = classifier.evaluate(testloader)

print(f"\nMiglioramento: {accuracy_before:.2f}% -> {accuracy_after:.2f}% (+{accuracy_after-accuracy_before:.2f}%)")

c:\Users\frogora\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\frogora\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Modello creato con 1024 -> 10 neuroni nel classificatore
=== ACCURACY PRIMA DEL FINE-TUNING ===


In [ ]:
#!pip install captum

import torch
import matplotlib.pyplot as plt
from captum.attr import LayerGradCam, LayerAttribution, Occlusion
import numpy as np

class ExplainabilityAnalyzer:
    """Classe per l'analisi di explainability dei modelli"""
    
    def __init__(self, model: nn.Module, device: torch.device):
        self.model = model
        self.device = device
        self.model.eval()
        
        # Identifica il target layer per GradCAM (ultimo layer convoluzionale)
        self.target_layer = self._find_target_layer()
        
        # Inizializza gli explainer
        self.gradcam = LayerGradCam(self.model, self.target_layer)
        self.occlusion = Occlusion(self.model)
        
    def _find_target_layer(self):
        """Trova l'ultimo layer convoluzionale per GradCAM"""
        # Per DenseNet121, l'ultimo layer convoluzionale è features[-1]
        return self.model.features[-1]
    
    def explain_with_gradcam(self, input_tensor: torch.Tensor, target_class: int = None) -> torch.Tensor:
        """Genera spiegazione con GradCAM"""
        input_tensor = input_tensor.to(self.device)
        
        if target_class is None:
            # Se non specificato, usa la predizione del modello
            with torch.no_grad():
                output = self.model(input_tensor)
                target_class = torch.argmax(output, dim=1).item()
        
        # Genera attributions
        attributions = self.gradcam.attribute(input_tensor, target=target_class)
        
        # Upsample alle dimensioni dell'input
        upsampled_attr = LayerAttribution.interpolate(
            attributions, input_tensor.shape[2:]
        )
        
        return upsampled_attr, target_class
    
    def explain_with_occlusion(self, input_tensor: torch.Tensor, target_class: int = None,
                              sliding_window_shapes: Tuple = (3, 15, 15),
                              strides: Tuple = (3, 8, 8)) -> torch.Tensor:
        """Genera spiegazione con Occlusion"""
        input_tensor = input_tensor.to(self.device)
        
        if target_class is None:
            with torch.no_grad():
                output = self.model(input_tensor)
                target_class = torch.argmax(output, dim=1).item()
        
        attributions = self.occlusion.attribute(
            input_tensor,
            strides=strides,
            target=target_class,
            sliding_window_shapes=sliding_window_shapes
        )
        
        return attributions, target_class
    
    def visualize_explanation(self, input_tensor: torch.Tensor, attributions: torch.Tensor,
                            predicted_class: int, true_class: int = None, method_name: str = ""):
        """Visualizza l'immagine originale e la saliency map"""
        
        # Prepara l'immagine per la visualizzazione
        img = input_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy()
        
        # Denormalizza l'immagine (ImageNet normalization)
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        # Prepara la saliency map
        saliency = attributions.squeeze().detach().cpu().numpy()
        if saliency.ndim == 3:
            saliency = saliency.mean(axis=0)
        
        # Normalizza la saliency map
        saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)
        
        # Crea la visualizzazione
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Immagine originale
        axes[0].imshow(img)
        axes[0].set_title("Immagine Originale")
        axes[0].axis('off')
        
        # Saliency map
        im = axes[1].imshow(saliency, cmap='hot')
        axes[1].set_title(f"Saliency Map - {method_name}")
        axes[1].axis('off')
        plt.colorbar(im, ax=axes[1])
        
        # Overlay
        axes[2].imshow(img, alpha=0.6)
        axes[2].imshow(saliency, cmap='hot', alpha=0.4)
        title = f"Overlay - Predetto: {predicted_class}"
        if true_class is not None:
            title += f", Vero: {true_class}"
        axes[2].set_title(title)
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    def comprehensive_analysis(self, input_tensor: torch.Tensor, true_class: int = None):
        """Esegue un'analisi completa con tutti i metodi disponibili"""
        
        # Ottieni predizione
        with torch.no_grad():
            output = self.model(input_tensor.to(self.device))
            probabilities = torch.softmax(output, dim=1)
            predicted_class = torch.argmax(output, dim=1).item()
            confidence = probabilities[0, predicted_class].item()
        
        print(f"Predizione: Classe {predicted_class} (Confidenza: {confidence:.3f})")
        if true_class is not None:
            print(f"Classe vera: {true_class}")
            print(f"Predizione {'CORRETTA' if predicted_class == true_class else 'ERRATA'}")
        
        # GradCAM
        print("\n--- Analisi GradCAM ---")
        gradcam_attr, _ = self.explain_with_gradcam(input_tensor, predicted_class)
        self.visualize_explanation(input_tensor, gradcam_attr, predicted_class, true_class, "GradCAM")
        
        # Occlusion
        print("\n--- Analisi Occlusion ---")
        occlusion_attr, _ = self.explain_with_occlusion(input_tensor, predicted_class)
        self.visualize_explanation(input_tensor, occlusion_attr, predicted_class, true_class, "Occlusion")
        
        return {
            'predicted_class': predicted_class,
            'confidence': confidence,
            'gradcam_attributions': gradcam_attr,
            'occlusion_attributions': occlusion_attr
        }

# Inizializza l'analyzer
model = classifier.get_model()
explainer = ExplainabilityAnalyzer(model, device)

print("Explainability Analyzer inizializzato con successo!")

In [ ]:
# 1. Downgrade numpy
#!pip install numpy==1.23.5 --force-reinstall

# 2. Reinstalla lime
#!pip install lime --force-reinstall

try:
    import lime
    from lime import lime_image
    from skimage.segmentation import mark_boundaries
    LIME_AVAILABLE = True
except ImportError:
    print("LIME non disponibile. Installazione...")
    # %pip install lime scikit-image
    # Poi riavviare il kernel
    LIME_AVAILABLE = False

class LIMEAnalyzer:
    """Classe per l'analisi LIME delle predizioni"""
    
    def __init__(self, model: nn.Module, device: torch.device):
        self.model = model
        self.device = device
        self.model.eval()
        
        if LIME_AVAILABLE:
            self.explainer = lime_image.LimeImageExplainer()
        else:
            print("LIME non disponibile. Installare con: pip install lime scikit-image")
    
    def _batch_predict(self, images):
        """Funzione per predizioni batch richiesta da LIME"""
        self.model.eval()
        
        # Converte le immagini in tensori
        batch_tensors = []
        for img in images:
            # LIME passa immagini RGB [0,1], dobbiamo normalizzarle per ImageNet
            img_tensor = torch.tensor(img).permute(2, 0, 1).float()
            
            # Applica la normalizzazione ImageNet
            normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            img_tensor = normalize(img_tensor)
            batch_tensors.append(img_tensor)
        
        batch = torch.stack(batch_tensors).to(self.device)
        
        with torch.no_grad():
            logits = self.model(batch)
            probs = torch.nn.functional.softmax(logits, dim=1)
        
        return probs.detach().cpu().numpy()
    
    def explain_instance(self, input_tensor: torch.Tensor, num_features: int = 5,
                        num_samples: int = 1000) -> Dict[str, Any]:
        """Genera spiegazione LIME per un'istanza"""
        
        if not LIME_AVAILABLE:
            print("LIME non disponibile.")
            return {}
        
        # Prepara l'immagine per LIME (RGB [0,1])
        img = input_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy()
        
        # Denormalizza l'immagine
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        # Ottieni la predizione
        with torch.no_grad():
            output = self.model(input_tensor.to(self.device))
            predicted_class = torch.argmax(output, dim=1).item()
        
        # Genera spiegazione LIME
        explanation = self.explainer.explain_instance(
            img,
            self._batch_predict,
            top_labels=1,
            hide_color=0,
            num_samples=num_samples
        )
        
        # Estrai immagine e mask
        temp, mask = explanation.get_image_and_mask(
            predicted_class,
            positive_only=True,
            num_features=num_features,
            hide_rest=False
        )
        
        return {
            'explanation': explanation,
            'image': temp,
            'mask': mask,
            'predicted_class': predicted_class,
            'original_image': img
        }
    
    def visualize_lime_explanation(self, lime_result: Dict[str, Any]):
        """Visualizza i risultati LIME"""
        
        if not lime_result:
            return
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Immagine originale
        axes[0].imshow(lime_result['original_image'])
        axes[0].set_title("Immagine Originale")
        axes[0].axis('off')
        
        # Maschera LIME
        axes[1].imshow(lime_result['mask'], cmap='gray')
        axes[1].set_title("Maschera LIME")
        axes[1].axis('off')
        
        # Overlay con boundaries
        overlay = mark_boundaries(lime_result['image'], lime_result['mask'])
        axes[2].imshow(overlay)
        axes[2].set_title(f"LIME - Classe Predetta: {lime_result['predicted_class']}")
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()

# Inizializza LIME analyzer se disponibile
if LIME_AVAILABLE:
    lime_analyzer = LIMEAnalyzer(model, device)
    print("LIME Analyzer inizializzato con successo!")
else:
    print("LIME Analyzer non disponibile - installare le dipendenze necessarie")

In [ ]:
from captum.attr import Occlusion

class TrustworthinessEvaluator:
    """Classe principale per la valutazione di trustworthiness del modello"""
    
    def __init__(self, classifier: MNISTClassifier, data_manager: DataManager):
        self.classifier = classifier
        self.data_manager = data_manager
        self.model = classifier.get_model()
        self.device = device
        
        # Inizializza gli analyzer
        self.explainer = ExplainabilityAnalyzer(self.model, self.device)
        if LIME_AVAILABLE:
            self.lime_analyzer = LIMEAnalyzer(self.model, self.device)
        
    def evaluate_sample_predictions(self, num_samples: int = 10):
        """Valuta predizioni su un campione casuale"""
        print(f"=== VALUTAZIONE SU {num_samples} CAMPIONI CASUALI ===")
        
        correct_predictions = 0
        total_samples = 0
        confidence_scores = []
        
        sample_loader = DataLoader(
            self.data_manager.testset, 
            batch_size=1, 
            shuffle=True
        )
        
        for i, (image, true_label) in enumerate(sample_loader):
            if i >= num_samples:
                break
                
            image, true_label = image.to(self.device), true_label.to(self.device)
            
            with torch.no_grad():
                output = self.model(image)
                probabilities = torch.softmax(output, dim=1)
                predicted_class = torch.argmax(output, dim=1).item()
                confidence = probabilities[0, predicted_class].item()
            
            is_correct = predicted_class == true_label.item()
            correct_predictions += is_correct
            total_samples += 1
            confidence_scores.append(confidence)
            
            status = "✓" if is_correct else "✗"
            print(f"Campione {i+1}: {status} Predetto: {predicted_class}, "
                  f"Vero: {true_label.item()}, Confidenza: {confidence:.3f}")
        
        accuracy = correct_predictions / total_samples
        avg_confidence = np.mean(confidence_scores)
        
        print(f"\nRisultati:")
        print(f"Accuracy: {accuracy:.3f} ({correct_predictions}/{total_samples})")
        print(f"Confidenza media: {avg_confidence:.3f}")
        
        return accuracy, avg_confidence
    
    def analyze_misclassified_example(self):
        """Trova e analizza un esempio mal classificato"""
        print("=== RICERCA ESEMPIO MAL CLASSIFICATO ===")
        
        for images, true_labels in self.data_manager.testloader:
            images, true_labels = images.to(self.device), true_labels.to(self.device)
            
            with torch.no_grad():
                outputs = self.model(images)
                predicted_classes = torch.argmax(outputs, dim=1)
            
            # Trova il primo esempio mal classificato nel batch
            for i in range(len(images)):
                if predicted_classes[i] != true_labels[i]:
                    # Estrai la singola immagine e label
                    image = images[i:i+1]  # Mantieni la dimensione batch
                    true_label = true_labels[i]
                    predicted_class = predicted_classes[i].item()
                    
                    print(f"Trovato esempio mal classificato:")
                    print(f"Classe vera: {true_label.item()}, Predetta: {predicted_class}")
                    
                    # Analisi completa dell'esempio
                    print("\n=== ANALISI EXPLAINABILITY ===")
                    results = self.explainer.comprehensive_analysis(image, true_label.item())
                    
                    # Analisi LIME se disponibile
                    if LIME_AVAILABLE:
                        print("\n=== ANALISI LIME ===")
                        lime_results = self.lime_analyzer.explain_instance(image)
                        self.lime_analyzer.visualize_lime_explanation(lime_results)
                    
                    return image, true_label.item(), predicted_class, results
        
        print("Nessun esempio mal classificato trovato nei primi batch!")
        return None, None, None, None
    
    def analyze_correct_example(self):
        """Trova e analizza un esempio correttamente classificato"""
        print("=== ANALISI ESEMPIO CORRETTAMENTE CLASSIFICATO ===")
        
        for images, true_labels in self.data_manager.testloader:
            images, true_labels = images.to(self.device), true_labels.to(self.device)
            
            with torch.no_grad():
                outputs = self.model(images)
                predicted_classes = torch.argmax(outputs, dim=1)
            
            # Trova il primo esempio correttamente classificato nel batch
            for i in range(len(images)):
                if predicted_classes[i] == true_labels[i]:
                    # Estrai la singola immagine e label
                    image = images[i:i+1]  # Mantieni la dimensione batch
                    true_label = true_labels[i]
                    predicted_class = predicted_classes[i].item()
                    
                    print(f"Esempio correttamente classificato:")
                    print(f"Classe: {true_label.item()}")
                    
                    # Analisi completa dell'esempio
                    results = self.explainer.comprehensive_analysis(image, true_label.item())
                    
                    return image, true_label.item(), predicted_class, results
        
        return None, None, None, None
    
    def compare_explanation_methods(self, image: torch.Tensor, true_class: int):
        """Confronta i diversi metodi di explainability sullo stesso esempio"""
        print("=== CONFRONTO METODI DI EXPLAINABILITY ===")
        
        # GradCAM
        gradcam_attr, pred_class = self.explainer.explain_with_gradcam(image)
        
        # Occlusion
        occlusion_attr, _ = self.explainer.explain_with_occlusion(image)
        
        # Visualizzazione comparativa
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # Prepara l'immagine
        img = image.squeeze().permute(1, 2, 0).detach().cpu().numpy()
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        # Immagine originale (prima riga)
        axes[0, 0].imshow(img)
        axes[0, 0].set_title(f"Originale\\nVero: {true_class}, Pred: {pred_class}")
        axes[0, 0].axis('off')
        
        # GradCAM
        gradcam_viz = gradcam_attr.squeeze().detach().cpu().numpy()
        if gradcam_viz.ndim == 3:
            gradcam_viz = gradcam_viz.mean(axis=0)
        gradcam_viz = (gradcam_viz - gradcam_viz.min()) / (gradcam_viz.max() - gradcam_viz.min() + 1e-8)
        
        axes[0, 1].imshow(gradcam_viz, cmap='hot')
        axes[0, 1].set_title("GradCAM")
        axes[0, 1].axis('off')
        
        axes[0, 2].imshow(img, alpha=0.6)
        axes[0, 2].imshow(gradcam_viz, cmap='hot', alpha=0.4)
        axes[0, 2].set_title("GradCAM Overlay")
        axes[0, 2].axis('off')
        
        # Occlusion
        occlusion_viz = occlusion_attr.squeeze().detach().cpu().numpy()
        if occlusion_viz.ndim == 3:
            occlusion_viz = occlusion_viz.mean(axis=0)
        occlusion_viz = (occlusion_viz - occlusion_viz.min()) / (occlusion_viz.max() - occlusion_viz.min() + 1e-8)
        
        axes[1, 0].axis('off')  # Vuoto per allineamento
        
        axes[1, 1].imshow(occlusion_viz, cmap='hot')
        axes[1, 1].set_title("Occlusion")
        axes[1, 1].axis('off')
        
        axes[1, 2].imshow(img, alpha=0.6)
        axes[1, 2].imshow(occlusion_viz, cmap='hot', alpha=0.4)
        axes[1, 2].set_title("Occlusion Overlay")
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return {
            'gradcam': gradcam_attr,
            'occlusion': occlusion_attr,
            'predicted_class': pred_class
        }

# Inizializza il valutatore di trustworthiness
trustworthiness_evaluator = TrustworthinessEvaluator(classifier, data_manager)

print("Trustworthiness Evaluator inizializzato con successo!")

In [ ]:
print("MNIST: 70.000 immagini di cifre scritte a mano (0-9), 28x28 pixel, 1 canale (convertito a 3 per DenseNet).")

# === VALUTAZIONE COMPLETA DEL MODELLO ===

print("VALUTAZIONE TRUSTWORTHINESS DEL MODELLO MNIST")
print("=" * 60)

# 1. Valutazione su campioni casuali
accuracy, avg_confidence = trustworthiness_evaluator.evaluate_sample_predictions(num_samples=20)

print(f"\nRIEPILOGO PERFORMANCE:")
print(f"   • Accuracy sui campioni: {accuracy:.1%}")
print(f"   • Confidenza media: {avg_confidence:.3f}")

# 2. Analisi di un esempio correttamente classificato
print(f"\nANALISI ESEMPIO CORRETTO:")
print("-" * 40)
correct_img, correct_true, correct_pred, correct_results = trustworthiness_evaluator.analyze_correct_example()

In [ ]:
for images, labels in testloader:
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    
    # Cerca il primo errore nel batch
    for i in range(len(images)):
        if preds[i] != labels[i]:
            print(f"Errore: Predetto {preds[i].item()}, Reale {labels[i].item()}")
            break
    else:
        continue  # Se non trova errori in questo batch, continua al prossimo
    break  # Se trova un errore, esce dal loop esterno

# 3. Analisi di un esempio mal classificato
print(f"\nANALISI ESEMPIO MAL CLASSIFICATO:")
print("-" * 40)
error_img, error_true, error_pred, error_results = trustworthiness_evaluator.analyze_misclassified_example()

if error_img is not None:
    print(f"\nCONFRONTO TRA PREDIZIONE CORRETTA E ERRATA:")
    print("-" * 50)
    
    # Confronta i metodi di explainability sull'esempio errato
    comparison_results = trustworthiness_evaluator.compare_explanation_methods(error_img, error_true)

In [ ]:
def simple_rule_based_classifier(img):
    img = img.squeeze().numpy()
    if img.sum() < 100:
        return 1
    else:
        return 0

class StabilityAnalyzer:
    """Classe per analizzare la stabilità delle spiegazioni"""
    
    def __init__(self, explainer: ExplainabilityAnalyzer):
        self.explainer = explainer
        
    def test_explanation_stability(self, image: torch.Tensor, num_runs: int = 5):
        """Testa la stabilità delle spiegazioni ripetendo l'analisi"""
        print(f"TEST STABILITÀ SPIEGAZIONI ({num_runs} runs)")
        print("-" * 40)
        
        gradcam_correlations = []
        occlusion_correlations = []
        
        # Esegui la prima spiegazione come riferimento
        ref_gradcam, pred_class = self.explainer.explain_with_gradcam(image)
        ref_occlusion, _ = self.explainer.explain_with_occlusion(image)
        
        # Appiattisci per calcolare correlazioni
        ref_gradcam_flat = ref_gradcam.flatten().detach().cpu().numpy()
        ref_occlusion_flat = ref_occlusion.flatten().detach().cpu().numpy()
        
        print(f"Classe predetta: {pred_class}")
        
        for run in range(1, num_runs):
            print(f"Run {run + 1}...", end=" ")
            
            # GradCAM
            gradcam_attr, _ = self.explainer.explain_with_gradcam(image)
            gradcam_flat = gradcam_attr.flatten().detach().cpu().numpy()
            gradcam_corr = np.corrcoef(ref_gradcam_flat, gradcam_flat)[0, 1]
            gradcam_correlations.append(gradcam_corr)
            
            # Occlusion
            occlusion_attr, _ = self.explainer.explain_with_occlusion(image)
            occlusion_flat = occlusion_attr.flatten().detach().cpu().numpy()
            occlusion_corr = np.corrcoef(ref_occlusion_flat, occlusion_flat)[0, 1]
            occlusion_correlations.append(occlusion_corr)
            
            print(f"GradCAM: {gradcam_corr:.3f}, Occlusion: {occlusion_corr:.3f}")
        
        # Statistiche
        gradcam_mean = np.mean(gradcam_correlations)
        gradcam_std = np.std(gradcam_correlations)
        occlusion_mean = np.mean(occlusion_correlations)
        occlusion_std = np.std(occlusion_correlations)
        
        print(f"\nRISULTATI STABILITÀ:")
        print(f"GradCAM - Media: {gradcam_mean:.3f} ± {gradcam_std:.3f}")
        print(f"Occlusion - Media: {occlusion_mean:.3f} ± {occlusion_std:.3f}")
        
        if gradcam_mean > 0.9 and occlusion_mean > 0.9:
            print("Spiegazioni STABILI (correlazione > 0.9)")
        elif gradcam_mean > 0.7 and occlusion_mean > 0.7:
            print("Spiegazioni MODERATAMENTE stabili (correlazione > 0.7)")
        else:
            print("Spiegazioni INSTABILI (correlazione < 0.7)")
        
        return {
            'gradcam_correlations': gradcam_correlations,
            'occlusion_correlations': occlusion_correlations,
            'gradcam_stats': (gradcam_mean, gradcam_std),
            'occlusion_stats': (occlusion_mean, occlusion_std)
        }

# Test della stabilità delle spiegazioni
if correct_img is not None:
    print(f"\nANALISI STABILITÀ DELLE SPIEGAZIONI")
    print("=" * 60)
    
    stability_analyzer = StabilityAnalyzer(explainer)
    stability_results = stability_analyzer.test_explanation_stability(correct_img, num_runs=5)

In [ ]:
errore_img = None
errore_label = None
errore_pred = None

for images, labels in testloader:
    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        # Cerca il primo errore nel batch
        for i in range(len(images)):
            if preds[i] != labels[i]:
                print(f"Errore: Predetto {preds[i].item()}, Reale {labels[i].item()}")
                errore_img = images[i:i+1]  # Mantieni dimensione batch
                errore_label = labels[i].item()
                errore_pred = preds[i].item()
                break
        
        # Se trovato un errore, esci dal loop esterno
        if errore_img is not None:
            break

class TrustworthinessMetrics:
    """Classe per calcolare metriche quantitative di trustworthiness"""
    
    def __init__(self, model: nn.Module, device: torch.device):
        self.model = model
        self.device = device
        
    def calculate_prediction_confidence_distribution(self, dataloader: DataLoader, max_samples: int = 1000):
        """Calcola la distribuzione delle confidenze delle predizioni"""
        print("ANALISI DISTRIBUZIONE CONFIDENZE")
        print("-" * 40)
        
        confidences = []
        correct_confidences = []
        incorrect_confidences = []
        
        self.model.eval()
        samples_processed = 0
        
        with torch.no_grad():
            for images, labels in dataloader:
                if samples_processed >= max_samples:
                    break
                    
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                probabilities = torch.softmax(outputs, dim=1)
                
                predicted_classes = torch.argmax(outputs, dim=1)
                max_probs = torch.max(probabilities, dim=1)[0]
                
                for i in range(len(images)):
                    confidence = max_probs[i].item()
                    confidences.append(confidence)
                    
                    if predicted_classes[i] == labels[i]:
                        correct_confidences.append(confidence)
                    else:
                        incorrect_confidences.append(confidence)
                
                samples_processed += len(images)
        
        # Statistiche
        print(f"Campioni analizzati: {len(confidences)}")
        print(f"Confidenza media: {np.mean(confidences):.3f}")
        print(f"Confidenza mediana: {np.median(confidences):.3f}")
        print(f"Deviazione standard: {np.std(confidences):.3f}")
        
        if correct_confidences and incorrect_confidences:
            print(f"Confidenza predizioni corrette: {np.mean(correct_confidences):.3f}")
            print(f"Confidenza predizioni errate: {np.mean(incorrect_confidences):.3f}")
            
            # Visualizzazione
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 2, 1)
            plt.hist(confidences, bins=30, alpha=0.7, color='blue', label='Tutte')
            plt.hist(correct_confidences, bins=30, alpha=0.7, color='green', label='Corrette')
            plt.hist(incorrect_confidences, bins=30, alpha=0.7, color='red', label='Errate')
            plt.xlabel('Confidenza')
            plt.ylabel('Frequenza')
            plt.title('Distribuzione Confidenze')
            plt.legend()
            
            plt.subplot(1, 2, 2)
            plt.boxplot([correct_confidences, incorrect_confidences], 
                       labels=['Corrette', 'Errate'])
            plt.ylabel('Confidenza')
            plt.title('Boxplot Confidenze')
            
            plt.tight_layout()
            plt.show()
        
        return {
            'all_confidences': confidences,
            'correct_confidences': correct_confidences,
            'incorrect_confidences': incorrect_confidences,
            'mean_confidence': np.mean(confidences),
            'confidence_std': np.std(confidences)
        }
    
    def evaluate_calibration(self, dataloader: DataLoader, max_samples: int = 1000, n_bins: int = 10):
        """Valuta la calibrazione del modello (reliability diagram)"""
        print("VALUTAZIONE CALIBRAZIONE MODELLO")
        print("-" * 40)
        
        all_confidences = []
        all_accuracies = []
        
        self.model.eval()
        samples_processed = 0
        
        with torch.no_grad():
            for images, labels in dataloader:
                if samples_processed >= max_samples:
                    break
                    
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                probabilities = torch.softmax(outputs, dim=1)
                
                predicted_classes = torch.argmax(outputs, dim=1)
                max_probs = torch.max(probabilities, dim=1)[0]
                
                confidences = max_probs.cpu().numpy()
                accuracies = (predicted_classes == labels).float().cpu().numpy()
                
                all_confidences.extend(confidences)
                all_accuracies.extend(accuracies)
                
                samples_processed += len(images)
        
        # Crea i bin per la calibrazione
        bin_boundaries = np.linspace(0, 1, n_bins + 1)
        bin_lowers = bin_boundaries[:-1]
        bin_uppers = bin_boundaries[1:]
        
        bin_confidences = []
        bin_accuracies = []
        bin_counts = []
        
        for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
            in_bin = np.logical_and(np.array(all_confidences) > bin_lower, 
                                   np.array(all_confidences) <= bin_upper)
            prop_in_bin = in_bin.mean()
            
            if prop_in_bin > 0:
                accuracy_in_bin = np.array(all_accuracies)[in_bin].mean()
                avg_confidence_in_bin = np.array(all_confidences)[in_bin].mean()
                bin_accuracies.append(accuracy_in_bin)
                bin_confidences.append(avg_confidence_in_bin)
                bin_counts.append(in_bin.sum())
            else:
                bin_accuracies.append(0)
                bin_confidences.append(0)
                bin_counts.append(0)
        
        # Expected Calibration Error (ECE)
        ece = 0
        for i in range(n_bins):
            if bin_counts[i] > 0:
                ece += (bin_counts[i] / len(all_confidences)) * abs(bin_confidences[i] - bin_accuracies[i])
        
        print(f"Expected Calibration Error (ECE): {ece:.4f}")
        
        # Visualizzazione reliability diagram
        plt.figure(figsize=(8, 6))
        plt.bar(range(n_bins), bin_accuracies, alpha=0.7, label='Accuracy', width=0.8)
        plt.plot(range(n_bins), bin_confidences, 'ro-', label='Confidence', linewidth=2)
        plt.plot([0, n_bins-1], [bin_lowers[0], bin_uppers[-1]], 'k--', label='Perfect Calibration')
        
        plt.xlabel('Confidence Bin')
        plt.ylabel('Accuracy / Confidence')
        plt.title(f'Reliability Diagram (ECE: {ece:.4f})')
        plt.legend()
        plt.xticks(range(n_bins), [f'{bin_lowers[i]:.1f}-{bin_uppers[i]:.1f}' for i in range(n_bins)], rotation=45)
        plt.tight_layout()
        plt.show()
        
        return {
            'ece': ece,
            'bin_accuracies': bin_accuracies,
            'bin_confidences': bin_confidences,
            'bin_counts': bin_counts
        }

# Valutazione quantitativa della trustworthiness
print("\nVALUTAZIONE QUANTITATIVA TRUSTWORTHINESS")
print("=" * 60)

metrics_evaluator = TrustworthinessMetrics(model, device)

# Analisi distribuzione confidenze
confidence_analysis = metrics_evaluator.calculate_prediction_confidence_distribution(testloader, max_samples=1000)

# Analisi calibrazione
calibration_analysis = metrics_evaluator.evaluate_calibration(testloader, max_samples=1000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
from torch import nn
from captum.attr import LayerGradCam, LayerAttribution, Occlusion

# Usa il modello già addestrato invece di crearne uno nuovo
model = classifier.get_model()
model.eval()

def show_saliency_on_image(input_tensor, attributions, title="Saliency Map"):
    """Funzione per visualizzare la saliency map sull'immagine"""
    
    # Prepara l'immagine per la visualizzazione
    img = input_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy()
    
    # Denormalizza l'immagine (ImageNet normalization)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    
    # Prepara la saliency map
    saliency = attributions.squeeze().detach().cpu().numpy()
    if saliency.ndim == 3:
        saliency = saliency.mean(axis=0)
    
    # Normalizza la saliency map
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)
    
    # Crea la visualizzazione
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Immagine originale
    axes[0].imshow(img)
    axes[0].set_title("Immagine Originale")
    axes[0].axis('off')
    
    # Saliency map
    im = axes[1].imshow(saliency, cmap='hot')
    axes[1].set_title("Saliency Map")
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1])
    
    # Overlay
    axes[2].imshow(img, alpha=0.6)
    axes[2].imshow(saliency, cmap='hot', alpha=0.4)
    axes[2].set_title(title)
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

# Verifica che errore_img esista
if 'errore_img' in locals() and errore_img is not None:
    target_layer = model.features[-1]
    gradcam = LayerGradCam(model, target_layer)
    occlusion = Occlusion(model)

    attributions = gradcam.attribute(errore_img, target=errore_label)
    upsampled_attr = LayerAttribution.interpolate(attributions, errore_img.shape[2:])
    show_saliency_on_image(errore_img, upsampled_attr, title=f"Grad-CAM (Predetto {errore_pred}, Reale {errore_label})")

    attr_occ = occlusion.attribute(
        errore_img,
        strides=(3, 8, 8),
        target=errore_label,
        sliding_window_shapes=(3, 15, 15)
    )
    show_saliency_on_image(errore_img, attr_occ, title=f"Occlusion (Predetto {errore_pred}, Reale {errore_label})")
else:
    print("Nessun esempio di errore trovato per l'analisi saliency.")

# === CONCLUSIONI E RIEPILOGO ===

print("\nRIEPILOGO ANALISI TRUSTWORTHINESS")
print("=" * 60)

print("RISULTATI PRINCIPALI:")
print(f"   1. Accuracy dopo fine-tuning: {accuracy_after:.2f}%")
print(f"   2. Expected Calibration Error: {calibration_analysis['ece']:.4f}")
print(f"   3. Confidenza media: {confidence_analysis['mean_confidence']:.3f}")

print(f"\nMETODOLOGIE IMPLEMENTATE:")
print(f"   • GradCAM: Evidenziazione regioni importanti tramite gradienti")
print(f"   • Occlusion: Analisi importanza tramite occlusione sistematica")
if LIME_AVAILABLE:
    print(f"   • LIME: Spiegazioni locali tramite perturbazioni")
print(f"   • Analisi stabilità: Consistenza delle spiegazioni")
print(f"   • Calibrazione: Affidabilità delle confidenze")

print(f"\nARCHITETTURA MODULARE:")
print(f"   • DataManager: Gestione dati e preprocessing")
print(f"   • MNISTClassifier: Fine-tuning e valutazione modello") 
print(f"   • ExplainabilityAnalyzer: Analisi XAI integrate")
print(f"   • TrustworthinessEvaluator: Valutazione complessiva")
print(f"   • StabilityAnalyzer: Test robustezza spiegazioni")
print(f"   • TrustworthinessMetrics: Metriche quantitative")

print(f"\nPROBLEMI RISOLTI:")
print(f"   • Ultimo layer non addestrato: RISOLTO con fine-tuning")
print(f"   • Accuracy ~0.08%: MIGLIORATO a {accuracy_after:.2f}%")
print(f"   • Struttura non modulare: RISOLTO con architettura a classi")
print(f"   • Mancanza di valutazione quantitativa: AGGIUNTO analisi metriche")

print(f"\nTRUSTWORTHINESS ASSESSMENT:")
if accuracy_after > 80:
    print("   ALTA: Modello affidabile per analisi XAI")
elif accuracy_after > 60:
    print("   MEDIA: Modello utilizzabile con cautela")
else:
    print("   BASSA: Modello necessita ulteriore training")

if calibration_analysis['ece'] < 0.1:
    print("   Ben calibrato: Confidenze affidabili")
else:
    print("   Mal calibrato: Confidenze da interpretare con cautela")

print(f"\nINSIGHTS FINALI:")
print(f"   • Il fine-tuning dell'ultimo layer è ESSENZIALE per XAI significativa")
print(f"   • L'architettura modulare facilita estensioni e manutenzione")
print(f"   • La valutazione quantitativa è cruciale per la trustworthiness")
print(f"   • GradCAM e Occlusion forniscono spiegazioni complementari")

print(f"\nPOSSIBILI ESTENSIONI:")
print(f"   • Implementazione di altri metodi XAI (SHAP, Integrated Gradients)")
print(f"   • Analisi su dataset più complessi")
print(f"   • Studio dell'interpretabilità cross-domain")
print(f"   • Sviluppo di metriche custom di trustworthiness")

print("\n" + "=" * 60)
print("ANALISI COMPLETATA CON SUCCESSO!")